# Introduction

#variance-covariance, #sbm, #frtb, #cva-frtb, #simm

This is a demonstration of the variance-covariance aggregation in atoti.

Run this notebook to start up a cube aggregating equity delta sensitivities into the SBM charge. The SBM methodology is implemented as described in the Consolidated Basel Framework Chapter MAR21: https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101. 

The variance-covariance method is utilized internally by organisations, as well as by the other regulatory methodologies, such as SIMM, CVA-FRTB to compute VaR-type measures from sensitivities.

Input data is sourced in CRIF-like format: https://www.isda.org/a/owEDE/risk-data-standards-v1-36-public.pdf

# Imports

In [1]:
from IPython.display import display, Markdown, Latex, Image
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", 300)
pd.options.display.float_format = '{:,.4f}'.format
diagram = {}

# Atoti

In [2]:
import atoti as av
from atoti.config import create_config
session = av.create_session(
    config='./configuration.yaml', port='53972', sampling_mode=av.sampling.FULL)

# Url

In [3]:
display(Markdown('[' + session.url+ ']('+ session.url+')'))

[http://localhost:53972](http://localhost:53972)

# Reading sensitivities

In [4]:
crif = pd.read_csv("data/smaller_data.csv")
crif = crif.append(pd.read_csv("data/bigger_data.csv"), ignore_index = True)
crif.head(5)

,TradeID,RiskType,Qualifier,Label2,AmountUSD,Bucket,PortfolioID
0,0,Risk_Equity,Wilmar International,REPO,"-10,332.0900",1,Smaller_Portfolio
1,1,Risk_Equity,Wilmar International,SPOT,"-3,641,606.4500",1,Smaller_Portfolio
2,2,Risk_Equity,China Minmetals,REPO,"-2,337.9000",3,Smaller_Portfolio
3,3,Risk_Equity,China Minmetals,SPOT,"-10,549.5400",3,Smaller_Portfolio
4,4,Risk_Equity,China Life Insurance,REPO,"-7,874.9400",4,Smaller_Portfolio


In [5]:
risks_store = session.read_pandas(crif,
                                  keys=['TradeID', 'PortfolioID','RiskType', 'Qualifier', 'Label2'],
                                  store_name="Risks",
                                  types={'Bucket': av.types.STRING})

cube = session.create_cube(risks_store)
lvl = cube.levels
m = cube.measures
h = cube.hierarchies

# Risk weights datastore

In [6]:
eq_delta_risk_weights_store = session.read_csv(
    "data/parameters/eq_delta_rw.csv",
    keys=["Bucket", "Label2"],
    types={"Bucket": av.types.STRING},
    store_name="RiskWeights"
)
risks_store.join(eq_delta_risk_weights_store)

In [7]:
eq_delta_risk_weights_store.head(5)

,,RW
Bucket,Label2,
1,SPOT,0.5500
2,SPOT,0.6000
3,SPOT,0.4500
4,SPOT,0.5500
5,SPOT,0.3000


# Weighted sensitivities

In [8]:
m['WS_k'] = av.agg.sum(m['AmountUSD.SUM']*m['RW.VALUE'],
                       scope=av.scope.origin("Bucket","Label2"))

# Bucket-level aggregation

## Risk factor correlations

The three cases for each pair of risk factors:

- Case 1: same name, different type: a single value -> 0.999
- Case 2: different name, same type: a single value depending on bucket, for example, 0.15
- Case 3: different name, different type: value depending on risk factor multiplier by 0.999, for example, 0.15 x 0.999

Example


| risk factors | name1-spot | name1-repo | name2-spot | name2-repo |
|------------|-------------|-------------|-------------|-------------|
| name1-spot | 1 |  |  |  |
| name1-repo | same_name_diff_type | 1 |  |  |
| name2-spot | rho_by_name | rho_by_name x type_multiplier | 1 |  |
| name2-repo | rho_by_name x type_multiplier | rho_by_name x type_multiplier | same_name_diff_type | 1 |

In [9]:
# risk factor = name + type
# correlations and multipliers:
same_risk_factor = 1.0
same_name_diff_type = 0.999
diff_type_multiplier = 0.999

In [10]:
# correlations by bucket - datastore creation
eq_delta_rho = session.read_csv(
    "data/parameters/eq_delta_rho.csv", keys=["Bucket"], types={"Bucket": av.types.STRING}, 
    store_name="RiskFactorCorrelations"
)
risks_store.join(eq_delta_rho)
eq_delta_rho.head(5)

,names_correlation
Bucket,
1,0.1500
2,0.1500
3,0.1500
4,0.1500
5,0.2500


## Method 1: materializing risk factor pairs

In [11]:
# Important:
# We join by POrtfolioID - hence  
# the combinations of risk factors are restricted to those that defined below PortfolioID

In [12]:
other_risk_factor_store = session.read_pandas(crif[['PortfolioID', 'Bucket', 'Qualifier', 'Label2']].rename(columns={'Qualifier': 'Other Qualifier', 'Label2': 'Other Label2'}), keys=[
                                              'PortfolioID', 'Bucket', 'Other Qualifier', 'Other Label2'], types={'Bucket': av.types.STRING}, store_name='OtherRiskFactor')
risks_store.join(other_risk_factor_store, mapping={
                 'PortfolioID': 'PortfolioID', 'Bucket': 'Bucket'})

In [13]:
# measure returning correlation
m['rho_kl'] = av.where(lvl['Qualifier'] == lvl['Other Qualifier'], 
                       av.where(lvl['Label2'] == lvl['Other Label2'], same_risk_factor, same_name_diff_type),
                       av.where(lvl['Label2'] == lvl['Other Label2'], m['names_correlation.VALUE'],
                                m['names_correlation.VALUE'] * diff_type_multiplier))

In [14]:
m['Other WS_k'] = av.at(m['WS_k'], {
                        lvl['Qualifier']: lvl['Other Qualifier'],
                        lvl['Label2']: lvl['Other Label2']})

m['K_b'] = av.agg.stop(av.where(lvl['Bucket'] == '11',
                                av.agg.sum(av.abs(m['WS_k']), av.scope.origin('Qualifier','Label2')),
                                av.sqrt(av.max(0, av.agg.sum(m['WS_k']
                                                             * m['Other WS_k'] * m['rho_kl'], scope = av.scope.origin('Qualifier','Other Qualifier','Label2','Other Label2'))))), at = [lvl['Bucket']])

## Method 2: optimized Kb calculation

Since many of the risk pairs share the same correlation value, it is possible to optimize the variance-covariance aggregation. The efficiency of this calculation is critical when cardinality of data along risk factor is high.

We decompose the formula into three components:
- contribution of pairs with both risk factors being spot - same correlation $\rho_{names}$ set per bucket,
- contribution of pairs with both risk factors being repo - same correlation $\rho_{names}$ set per bucket,
- contribution of pairs where one risk factor is spot, another one is repo - same correlation $\rho_{names} \cdot 0.999$. We'd need to account that for some of the pairs where one risk factor is spot, one is repo - some of the names will match and need to be correlated at 0.999


Let's start with the pairs where both risk factors are either spot or repo. Their contribution can be rewritten:

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \sum_k WS_k^2 + \rho_{kl} \cdot \left(\left( \sum_k WS_k \right)^2 - \sum_k WS_k^2  \right)$$

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = (1-\rho_{kl}) \cdot \sum_k WS_k^2 + \rho_{kl} \cdot \left( \sum_k WS_k \right)^2  $$

In [15]:
m['sum squares'] = av.agg.square_sum(m['WS_k'], scope = av.scope.origin('Qualifier'))
m['reduced formula'] = (1-m['names_correlation.VALUE'])*m['sum squares'] + m['names_correlation.VALUE']* av.pow(m['WS_k'],2.0)

# contribution of pairs having only spot risk factors:
m['spot contribution'] = av.filter(m['reduced formula'] , lvl['Label2']=="SPOT")
# contribution of pairs having only repo risk factors:
m['repo contribution'] = av.filter(m['reduced formula'] , lvl['Label2']=="REPO")

For pairs of risk factors where one risk factor belongs to "SPOT" and the other belongs to "REPO", the measure "sum WS_repo WS_spot" performs this matrix multiplication for each bucket: 

$$\vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}}$$

where J - is a matrix of ones. THe above result can be multiplied by $\rho_{kl}$ for a given bucket, corrected by the factor reflecting that both risk factors belong to different types and different names: $\rho_{kl} = \rho_{names} * 0.999$. 

$$\vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} \cdot \rho_{names} \cdot 0.999$$

SOme of the pairs may actually contain matching equity names, which must be correlated at 0.999. Their contribution is corrected by adding:

$$(0.999 - \rho_{names}) \sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}}$$


In [16]:
m['WS_spot'] = av.filter(m['WS_k'], lvl['Label2']=="SPOT")
m['WS_repo'] = av.filter(m['WS_k'], lvl['Label2']=="REPO")

m['WS_spot_vector'] = av.parent_value(av.agg._vector(m['WS_spot'], av.scope.origin('Qualifier')), on_hierarchies=[h['Qualifier']])
m["sum WS_repo J WS_spot"] = av.agg.sum(av.array.sum(m['WS_spot_vector']*m['WS_repo']), av.scope.origin('Qualifier'))

In [17]:
# this query visualized interim results
cube.query(m['WS_spot_vector'], m['WS_repo'], m["sum WS_repo J WS_spot"], levels = [lvl['Bucket'], lvl['Qualifier']])

WS_spot_vector  \
Bucket Qualifier                                                                                                                            
1      Wilmar International                                                                                         [-4005767.0950000007]   
10     AB Volvo                                                                       [-2422152.722, -1961608.41, -879956.03, -356670.49]   
       Anheuser-Busch InBev                                                           [-2422152.722, -1961608.41, -879956.03, -356670.49]   
       Christian Dior                                                                 [-2422152.722, -1961608.41, -879956.03, -356670.49]   
       Danone                                                                         [-2422152.722, -1961608.41, -879956.03, -356670.49]   
...                                                                                                                                   ...   
9      Gazprom               [1196946.758, 108348.45, 156175.15199999997, 1095322.2, 795571.938, 1930220.8939999999, -3793340.8379999995]   
       JBS S.A.              [1196946.758, 108348.45, 156175.15199999997, 1095322.2, 795571.938, 1930220.8939999999, -3793340.8379999995]   
       Lukoil                [1196946.758, 108348.45, 156175.15199999997, 1095322.2, 795571.938, 1930220.8939999999, -3793340.8379999995]   
       SAIC Motor            [1196946.758, 108348.45, 156175.15199999997, 1095322.2, 795571.938, 1930220.8939999999, -3793340.8379999995]   
       Severstal             [1196946.758, 108348.45, 156175.15199999997, 1095322.2, 795571.938, 1930220.8939999999, -3793340.8379999995]   

                              WS_repo  sum WS_repo J WS_spot  
Bucket Qualifier                                              
1      Wilmar International -113.6530       455,267,407.5904  
10     AB Volvo                7.1048       -39,931,730.1899  
       Anheuser-Busch InBev -374.0586     2,102,354,505.1760  
       Christian Dior         64.3994      -361,949,592.5562  
       Danone                -17.4460        98,053,282.9768  
...                               ...                    ...  
9      Gazprom                -2.1799        -3,246,463.7730  
       JBS S.A.              138.8859       206,835,040.4175  
       Lukoil               -156.4048      -232,924,966.8346  
       SAIC Motor            212.1448       315,935,488.0594  
       Severstal             -15.1826       -22,610,574.5807  

[1044 rows x 3 columns]

In [18]:
######### Below formula will multiply by null - might be a bad practice??

In [19]:
# cross product of weighted sensitivities for the names, having both spot and repo sensitivities:
m['sum WS repo and spot'] = av.agg.sum(m['WS_spot']*m['WS_repo'], scope = av.scope.origin('Qualifier'))

In [20]:
# final contribution of the pairs where one risk factor is spot, one is repo
m["cross repo spot contribution"] = m['names_correlation.VALUE']*m["sum WS_repo J WS_spot"]+(0.999- m['names_correlation.VALUE'])* m['sum WS repo and spot']

In [21]:
# Hence the formula for Kb can be repre

In [22]:
m['K_b alternative'] = av.sqrt(av.max(0, m['spot contribution'] + m['repo contribution'] + 2*m["cross repo spot contribution"]))

In [23]:
cube.query(m['K_b'], m['K_b alternative'], levels = [lvl['PortfolioID'], lvl['Bucket']])

K_b  K_b alternative
PortfolioID       Bucket                                
Bigger_Portfolio  1      2,002,940.3172   2,002,940.3172
                  10     1,819,315.9010   1,819,315.9185
                  11     6,418,348.7082   3,409,711.0045
                  12       286,992.4354     286,992.4354
                  2      1,247,249.6016   1,247,248.4588
                  3          4,757.8031       4,757.8031
                  4        758,465.4868     758,465.4930
                  5        456,861.0617     456,861.0687
                  7        932,682.8928     932,682.9003
                  8      3,937,788.9195   3,937,789.0203
                  9      2,234,809.9610   2,234,809.9651
Smaller_Portfolio 1      2,002,940.3172   2,002,940.3172
                  10     1,819,315.9010   1,819,315.9185
                  11     6,418,348.7082   3,409,711.0045
                  12       286,992.4354     286,992.4354
                  3          4,757.8031       4,757.8031
                  4        758,465.4868     758,465.4930
                  5        456,861.0617     456,861.0687
                  7        932,682.8928     932,682.9003
                  8      3,937,788.9195   3,937,789.0203
                  9      2,234,809.9610   2,234,809.9651

# Cross-bucket aggregation

## Bucket correlations

In [24]:
eq_delta_buckets_correlations = session.read_csv(
    "data/parameters/eq_delta_gamma.csv", keys=["Bucket", "Other Bucket"], types = {"Bucket": av.types.STRING, "Other Bucket": av.types.STRING}, store_name="eq_delta_corr_outer"
)
risks_store.join(eq_delta_buckets_correlations)

## Aggregating across buckets

In [25]:
# 21.4(5)(a):
m['WS_b'] = m['WS_k']
m['WS_c'] = av.at(m['WS_k'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum WSb WSc gamma'] = av.agg.square_sum(m['K_b'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['WS_b'] * m['WS_c']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

# 21.4(5)(b):
m['S_b'] = av.max(av.min(m['WS_k'], m['K_b']), -1.0 * m['K_b'])
m['S_c'] = av.at(m['S_b'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum Sb Sc gamma'] = av.agg.square_sum(m['K_b'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['S_b'] * m['S_c']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

m['Delta Margin'] = av.where(m['sum Kb2 + sum sum WSb WSc gamma'] > 0,
                             av.sqrt(m['sum Kb2 + sum sum WSb WSc gamma']),
                             av.sqrt(m['sum Kb2 + sum sum Sb Sc gamma']))

In [26]:
cube.query(m['Delta Margin'], levels = [lvl['PortfolioID']], timeout=90)

RuntimeError: ('Request failed', {'status': 'error', 'error': {'errorChain': [{'type': 'OutOfMemoryError', 'message': 'Java heap space'}, {'type': 'OutOfMemoryError'}, {'type': 'QueryException', 'message': 'Could not execute query at epoch 5. : ActivePivotSyncActionQuery: action=MdxAction (mdx=SELECT [Measures].[Delta Margin] ON COLUMNS, NON EMPTY [Hierarchies].[PortfolioID].[PortfolioID].Members ON ROWS FROM [Risks]), argument=com.quartetfs.pivot.mdx.impl.BasicSelectExecutor@2031aa7f'}, {'type': 'MdxException', 'message': 'Error with the following query: SELECT [Measures].[Delta Margin] ON COLUMNS, NON EMPTY [Hierarchies].[PortfolioID].[PortfolioID].Members ON ROWS FROM [Risks]'}, {'type': 'BadArgumentException', 'message': '[400] com.quartetfs.pivot.mdx.MdxException: Error with the following query: SELECT [Measures].[Delta Margin] ON COLUMNS, NON EMPTY [Hierarchies].[PortfolioID].[PortfolioID].Members ON ROWS FROM [Risks]'}], 'stackTrace': 'com.qfs.fwk.services.BadArgumentException: 

In [30]:
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Smaller_Portfolio", timeout=90)

,Delta Margin
0,"8,951,153.7749"


In [27]:
# Optimized calculation

# 21.4(5)(a):
m['sum Kb2 + sum sum WSb WSc gamma alternative'] = av.agg.square_sum(m['K_b alternative'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['WS_b'] * m['WS_c']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

# 21.4(5)(b):
m['S_b alternative'] = av.max(av.min(m['WS_k'], m['K_b alternative']), -1.0 * m['K_b alternative'])
m['S_c alternative'] = av.at(m['S_b alternative'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum Sb Sc gamma alternative'] = av.agg.square_sum(m['K_b alternative'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['S_b alternative'] * m['S_c alternative']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

m['Delta Margin alternative'] = av.where(m['sum Kb2 + sum sum WSb WSc gamma alternative'] > 0,
                             av.sqrt(m['sum Kb2 + sum sum WSb WSc gamma alternative']),
                             av.sqrt(m['sum Kb2 + sum sum Sb Sc gamma alternative']))

In [28]:
cube.query(m['Delta Margin alternative'], levels = [lvl['PortfolioID']], timeout=90)

,Delta Margin alternative
PortfolioID,
Bigger_Portfolio,"7,341,289.6650"
Smaller_Portfolio,"6,843,541.6738"


# Timeit

In [ ]:
risks_store.load_pandas(crif)

In [29]:
%%timeit
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Smaller_Portfolio", timeout=90)

76.2 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Bigger_Portfolio", timeout=90)

RuntimeError: ('Request failed', {'status': 'error', 'error': {'errorChain': [{'type': 'OutOfMemoryError', 'message': 'Java heap space'}, {'type': 'OutOfMemoryError'}, {'type': 'QueryException', 'message': 'Could not execute query at epoch 5. : ActivePivotSyncActionQuery: action=MdxAction (mdx=SELECT [Measures].[Delta Margin] ON COLUMNS FROM (SELECT ([Hierarchies].[PortfolioID].[ALL].[AllMember].[Bigger_Portfolio]) ON COLUMNS FROM [Risks])), argument=com.quartetfs.pivot.mdx.impl.BasicSelectExecutor@1b7bc63c'}, {'type': 'MdxException', 'message': 'Error with the following query: SELECT [Measures].[Delta Margin] ON COLUMNS FROM (SELECT ([Hierarchies].[PortfolioID].[ALL].[AllMember].[Bigger_Portfolio]) ON COLUMNS FROM [Risks])'}, {'type': 'BadArgumentException', 'message': '[400] com.quartetfs.pivot.mdx.MdxException: Error with the following query: SELECT [Measures].[Delta Margin] ON COLUMNS FROM (SELECT ([Hierarchies].[PortfolioID].[ALL].[AllMember].[Bigger_Portfolio]) ON COLUMNS FROM [R

In [32]:
%%timeit
cube.query(m['Delta Margin alternative'], condition = lvl['PortfolioID']=="Smaller_Portfolio")

61.1 ms ± 4.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
cube.query(m['Delta Margin alternative'], condition = lvl['PortfolioID']=="Bigger_Portfolio")

54.2 ms ± 5.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
